In [68]:
import tensorflow.compat.v1 as tf
import os
import sys
import subprocess
import numpy as np
import glob
import scipy.io as spio
tf.disable_v2_behavior()

In [69]:
print(tf.__version__)

2.1.0


In [36]:
def Convert2PB_tf_compat():
    trained_checkpoint_prefix = "model_512x32_lr_0_008_20200110"
    saved_model_path = '512x32_to_tflite_test2'
    export_dir = "export_dir"

    print(export_dir)

    graph = tf.Graph()
    with tf.compat.v1.Session(graph=graph) as sess:
        # Restore from checkpoint
        loader = tf.compat.v1.train.import_meta_graph(saved_model_path + "/"+ trained_checkpoint_prefix + '.meta')
        loader.restore(sess, saved_model_path)

        # Export checkpoint to SavedModel
        builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
        builder.add_meta_graph_and_variables(sess,
                                             [tf.saved_model.TRAINING, tf.saved_model.SERVING],
                                             strip_default_attrs=True)
        builder.save()            

In [37]:
def Convert2PB_Ori(meta_path, dir_name, output_node_name, output_pb_name):
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        # Restore the graph
        saver = tf.train.import_meta_graph(meta_path)

        # Load weights
        saver.restore(sess,tf.train.latest_checkpoint(dir_name))
        
        graph_def = tf.get_default_graph().as_graph_def()
        
        # Freeze the graph
        frozen_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            graph_def,
            [output_node_name])
        #'gradients_2/softmax_cross_entropy_with_logits_2_grad/LogSoftmax'
        # Save the frozen graph
        with open(dir_name+"/"+output_pb_name, 'wb') as f:
          f.write(frozen_graph_def.SerializeToString())
    

In [38]:
# Convert2PB_Ori("../TrainedModels/512x32_20200116_23_23_34/model_512x32_20200116_23_23_34.meta", \
#                "../TrainedModels/512x32_20200116_23_23_34","gradients/softmax_cross_entropy_with_logits_grad/LogSoftmax", \
#               "output_512x32.pb")

In [39]:
#gradients/softmax_cross_entropy_with_logits_grad/LogSoftmax
Convert2PB_Ori("../TrainedModels/256x64_20200117_04_04_38/model_256x64_20200117_04_04_38.meta", \
               "../TrainedModels/256x64_20200117_04_04_38",
               "gradients_1/softmax_cross_entropy_with_logits_1_grad/LogSoftmax", \
               "output_256x64_conversion2.pb")

#"gradients_1/softmax_cross_entropy_with_logits_1_grad/LogSoftmax" (256x64)
#"gradients/softmax_cross_entropy_with_logits_grad/LogSoftmax" (512x32) input:Placeholder

INFO:tensorflow:Restoring parameters from ../TrainedModels/256x64_20200117_04_04_38\model_256x64_20200117_04_04_38
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


In [78]:
"""
tflite_convert \
  --output_file=/tmp/foo.tflite \
  --graph_def_file=/tmp/some_quantized_graph.pb \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays=input \
  --output_arrays=MobilenetV1/Predictions/Reshape_1 \
  --mean_values=128 \
  --std_dev_values=127
  --default_ranges_min=0 \
  --default_ranges_max=6 \
"""
def Convert2PB_with_TFLite_Converter(pb_name, \
                                     output_lite_name, \
                                     input_node_array, \
                                     output_node_array, \
                                     meanValue,         \
                                     stddevValue,\
                                     maxvalue,
                                     minvalue):
    seperator = ' '
    argvlist = [
          "tflite_convert "
          "--output_file={}".format(output_lite_name),
          "--graph_def_file={}".format(pb_name),
          "--input_format=TENSORFLOW_GRAPHDEF", \
          "--output_format=TFLITE", \
          "--inference_type=QUANTIZED_UINT8",
          "--input_type=QUANTIZED_UINT8",
#           "--input_type=QUANTIZED_UINT8",
          "--input_arrays={}".format(input_node_array),
          "--output_arrays={}".format(output_node_array),
          "--mean_values={}".format(meanValue),
          "--std_dev_values={}".format(stddevValue),
          "--default_ranges_max={}".format(maxvalue),
          "--default_ranges_min={}".format(minvalue)
          
    ]
#     seperator.join("tflite_convert")
    run_command = seperator.join(argvlist)
    print(run_command)
    print("\n")
    subprocess.call(run_command)
#     exec("tflite_convert")

In [79]:
_max = str(sys.maxsize)
_min = 0 #-sys.maxsize -1
Convert2PB_with_TFLite_Converter("../TrainedModels/512x32_20200116_23_23_34/output_512x32.pb" , \
                                 "../TrainedModels/512x32_20200116_23_23_34/tflite_512x32_gen3.tflite",\
                                 "Placeholder",\
                                 "gradients/softmax_cross_entropy_with_logits_grad/LogSoftmax",\
                                 "128",\
                                 "127",
                                 255,
                                 0
                                 )

tflite_convert --output_file=../TrainedModels/512x32_20200116_23_23_34/tflite_512x32_gen3.tflite --graph_def_file=../TrainedModels/512x32_20200116_23_23_34/output_512x32.pb --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE --inference_type=QUANTIZED_UINT8 --input_type=QUANTIZED_UINT8 --input_arrays=Placeholder --output_arrays=gradients/softmax_cross_entropy_with_logits_grad/LogSoftmax --mean_values=128 --std_dev_values=127 --default_ranges_max=255 --default_ranges_min=0




In [50]:
# short way for output tflite???
def Convert2TFLite(saved_model_dir,tflitefile):
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    tflite_model = converter.convert()
    open(saved_model_dir+"/"+tflitefile, "wb").write(tflite_model)

In [51]:
def output_tb_for_tensorboard():
    sess = tf.Session()
    tf.train.import_meta_graph("..\\ExpResults\\256x64\\model_512x32_lr_0_008_20200110.meta")
    FileWriter("__tb", sess.graph)

In [80]:
def genWeightFromTFLite(tflite_file, ouput_mat_file):
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=tflite_file)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()


    # get details for each layer
    all_layers_details = interpreter.get_tensor_details() 
    

#     for layer in all_layers_details:
#         print("layer name={}".format(layer['name']))
#         print("layer shape={}".format(layer['shape']))
#         data=interpreter.get_tensor(layer['index'])
#         print("data is: \n{}".format(data))
    

    weight1 = []
    weight2 = []
    weight3 = []
    bias1 = []
    bias2= []
    bias3 = []
    for layer in all_layers_details:
        if layer['name']=="MatMul_1_bias":
            bias1 = interpreter.get_tensor(layer['index'])

        if layer['name']=="MatMul_2_bias":
            bias2 = interpreter.get_tensor(layer['index'])

        if layer['name']=="MatMul_bias":
             bias3 = interpreter.get_tensor(layer['index'])

        if layer['name']=="Variable/transpose":
            w1_tranpose = np.array(interpreter.get_tensor(layer['index']))
            weight1 = w1_tranpose.T

        if layer['name']=="Variable_1/transpose":
            w2_tranpose = np.array(interpreter.get_tensor(layer['index']))
            weight2 = w2_tranpose.T

        if layer['name']=="Variable_2/transpose":
            w3_tranpose = np.array(interpreter.get_tensor(layer['index']))
            weight3 = w3_tranpose.T

    print("w1.shape = {},\n w2.shape = {},\n w3.shape = {}\n".format(weight1.shape, weight2.shape, weight3.shape))

    spio.savemat(ouput_mat_file,\
                {'w1': weight1,'w2':weight2,'w3':weight3,\
                'b1':bias1,'b2':bias2,'b3':bias3})

In [81]:
genWeightFromTFLite("../TrainedModels/512x32_20200116_23_23_34/tflite_512x32_gen3.tflite",
                    "../TrainedModels/512x32_20200116_23_23_34/512x32_gen3.mat")

w1.shape = (40, 512),
 w2.shape = (512, 32),
 w3.shape = (32, 2)

